# Notebook run - Main file to run the translation approach. 

## Import libraries

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0";  

In [2]:
from collections import namedtuple
import os
import pandas as pd
from models.languageModels.translationModels import *
from models.learningModels import learningKerasModels
from utils.utils import *
from utils import generators
import tensorflow.keras as keras
from tensorflow.keras import optimizers
from collections import defaultdict
import tensorflow as tf
from utils import jiwer
from utils import evaluation
from utils.accuracy import accuracy
import mainp
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import single_meteor_score
import nltk
import copy
nltk.download('wordnet')

Using TensorFlow backend.
[nltk_data] Downloading package wordnet to /home/jota/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jota/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
print(tf.__version__)

1.13.2


In [4]:
from keras.backend.tensorflow_backend import set_session

config = tf.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth = True

set_session(tf.Session(config=config))

### Define all parameters

In [5]:
Args = namedtuple('Args', 'Tx Ty denseUnits encoder_unitLow_size encoder_unitHigh_size embedding_size vocab_out_size dropout recurrent_dropout inputShape annotations solver type batchSize epochs nIters lr momentum decay wDecay pathdata dataNLP rUnit path2save name FeedConnection nFilters')

In [6]:
args = Args(Tx=30, 
            Ty=53,
            denseUnits=512,
            encoder_unitLow_size=128,
            encoder_unitHigh_size=278,
            embedding_size=300,
            vocab_out_size=3003,
            dropout=0.4,
            recurrent_dropout=0.4,
            inputShape=[30, 227, 227, 3],
            annotations='../Datasets/Pre-procesados/SLR/RWTH-Phoenix/phoenixT/phoenix-2014-T.v3/PHOENIX-2014-T-release-v3/PHOENIX-2014-T/annotations/',
            solver='sgd',
            type='/ppm/',
            batchSize=1,
            epochs=20,
            nIters=15.0,
            lr=0.001,
            momentum=0.99,
            decay=0.1,
            wDecay=0.0005,
            pathdata=os.getcwd()+'/results/dataTrain_phoenix_227_227_30/',
            dataNLP=os.getcwd()+'/results/dataTrain_phoenix_sentences/',
            rUnit='LSTM',
            path2save=os.getcwd()+'/results/translations/',
            name='Bottom',
            FeedConnection=True,
            nFilters=60)

### Training ...

In [7]:
#---------------------------Prepare data for generators--------------------------#
print('Preparing data ..', args.type)
partition = defaultdict(list)
for f in ['train', 'test', 'dev']:
        path = args.pathdata+f+args.type+'/'
        for v in os.listdir(path):
            partition[f].append(path+v)

Preparing data .. /ppm/


In [8]:
model, encoder_model, decoder_model = mainp.main(args, partition)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30, 227, 227, 0                                            
__________________________________________________________________________________________________
conv1 (Conv3D)                  (None, 28, 225, 225, 2624        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalizationV1) (None, 28, 225, 225, 128         conv1[0][0]                      
__________________________________________________________________________________________________
activation (Activatio

### Generate the translations and save them!

In [9]:
print('Video to text with greedy search ...')
if args.name=='Top':
    print(args.name, ' Model')
    results = evaluation.video2textTOP(model, encoder_model, decoder_model, args, partition, name='PhoenixTop6060PPM',search='Greedy') 
elif args.name=='Bottom':
    print(args.name, ' Model')
    results = evaluation.video2textBottom(model, encoder_model, decoder_model, args, partition, name='PHOENIX-RGB-LSTM-MIDDLE-30-60', search='Greedy') 
elif args.name=='Double':
    print(args.name, ' Model')
    results = evaluation.video2textDouble(model, encoder_model, decoder_model, args, partition, name='PHOENIX-FLOW-LSTM-DOUBLE-60-256', search='Greedy')

Video to text with greedy search ...
Bottom  Model
train  data
video:  09December_2011_Friday_tagesschau-8439.npy
am tag regnet es im süden und südosten noch kräftig sonst ist es meist trocken <eos>
video:  06May_2011_Friday_tagesschau-6428.npy
und nun die wettervorhersage für morgen samstag den fünfundzwanzigsten januar <eos>
video:  01July_2010_Thursday_tagesschau-8791.npy
am tag regnet es im süden und südosten noch kräftig sonst ist es meist trocken <eos>
video:  10April_2010_Saturday_tagesschau-4136.npy
und nun die wettervorhersage für morgen samstag den dreißigsten februar <eos>
video:  10December_2011_Saturday_tagesschau-5415.npy
im norden und nordwesten bleibt es meist trocken <eos>
video:  17February_2010_Wednesday_heute-1487.npy
am tag regnet es im süden und südosten noch kräftig sonst ist es meist trocken <eos>
video:  12November_2009_Thursday_tagesschau-6773.npy
am tag regnet es im süden und südosten noch kräftig sonst ist es meist trocken <eos>
video:  01June_2011_Wednesday

In [ ]:
evaluation.metrics(copy.deepcopy(results), args)        

In [11]:
model.save_weights(os.getcwd()+"/models/weights/PHOENIX_30_60_RGB_modelRGB_"+args.name+'_'+args.rUnit+".h5")

In [12]:
encoder_model.save_weights(os.getcwd()+"/models/weights/PHOENIX_30_60_RGB_EncmodelRGB_"+args.name+'_'+args.rUnit+".h5")

In [13]:
decoder_model.save_weights(os.getcwd()+"/models/weights/PHOENIX_30_60_RGB_DecmodelRGB_"+args.name+'_'+args.rUnit+".h5")